In [4]:
# safe divide function to stop zero counts from causing problems. This is zero counts for number of words
# which can happen sometimes when texts are small, texts are empty, or content words are removed from small texts
def safe_divide(a, b): #function calls two arguments. Here it will be a POS count and number of words
    if b != 0: #
        return a/b
    else:
        return 0

In [3]:
import os
# Change to the working directory
os.chdir('/Users/zepedalab/Desktop/MATHia_NLP_YB')

#Lab wd:/Users/zepedalab/Desktop/MATHia_NLP_YB
#Home wd: /Users/yuchengbao/Desktop/ALL Lab/MATHia_NLP

import pandas as pd
import re
import spacy
!python3 -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

/Users/zepedalab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


/Users/zepedalab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.8 MB/s  0:00:00eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [24]:
#import the dataset
text_df = pd.read_excel('/Users/zepedalab/Desktop/MATHia_NLP_YB/E1C3 Codes (COMPLETE).xlsx')

#home wd: /Users/yuchengbao/PycharmProjects/JupyterProject/models/Practice Dataset.xlsx
#lab wd: /Users/zepedalab/Desktop/MATHia_NLP_YB/E1C3 Codes (COMPLETE).xlsx

In [33]:
# pre-process the text data (text_df)
text_df['input'] = text_df['input'].astype(str) # convert the text input to string type
text_df['input'] = text_df['input'].str.lower() # convert the text to lower case

text_df_3 = text_df[text_df['input'].str.len() >= 3] #can adjust the threshold here
short_input_df = text_df[text_df['input'].str.len() < 3]


In [34]:
short_input_df

,Anon.Student.Id,input,Invalid Response,Self-Efficacy,Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost \n(outside effort cost),Effort,Extrinsic Rewards/ Self-Consequating,Environmental Control,...,Fear of Failure,Attention,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Notes
1,stu_GDLLQ68531,no,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,NaN
5,stu_TMJAI99056,nn,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
6,stu_QRVFN34916,e,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
11,stu_VLXIG01010,k,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
13,stu_LHFEX95759,no,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,sch_NQNWK22810,..,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
422,sch_NQNWK22810,y\,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
425,sch_NQNWK22810,.,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
441,sch_NQNWK22810,.,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN


In [8]:
#Method 1: detect the gibberish and cross-check with the human labels
#from langdetect import detect_langs

#Method 2: gibberish detector based on training model

from gibberish_detector import detector
#os.chdir('/Users/yuchengbao/Desktop/ALL Lab/MATHia_NLP')
#os.system('gibberish-detector train big.txt > gibberish-detector.model')
Detector = detector.create_from_model('/Users/zepedalab/PycharmProjects/MATHia-NLP-YB/gibberish-detector.model')
#lab wd: /Users/zepedalab/PycharmProjects/MATHia-NLP-YB/gibberish-detector.model

In [35]:
#using detector to initially test the gibberish
# Create an empty dataframe to store gibberish rows
gibberish_df = pd.DataFrame(columns=text_df_3.columns)

# Iterate through rows and separate gibberish rows
for index, row in text_df_3.iterrows():
    try:
        # Check if the row is gibberish
        is_gibberish = Detector.is_gibberish(row['input'])

        # If it is gibberish, add to the gibberish_df and remove from text_df_3
        if is_gibberish:
            gibberish_df = pd.concat([gibberish_df, row.to_frame().T], ignore_index=True)
            text_df_3 = text_df_3.drop(index)

        print(row['input'], is_gibberish)

    except Exception as e:
        print(f"Error processing row {index}: {e}")


that i will awaly do better then last time False
nan False
nan False
. jjjnnk.nj..mi,.mk.ki.kii..l.kin. True
nan False
focus in classs or ask for help in work from the teacher or ask the fellow classmates. False
i am and i was hecking my work and if it didnt make since i would use the hit to help and solve it. False
i motivated myself by thinking about what would happen if i fail. False
less work to do after False
i worked hard and kept going. False
i worked hard and try to get it done False
idk so i can get out of school False
i just took my time and focus False
i just did it. False
keep tring?.. False
one of the many ways that i motivate myself is by saying that it will be ok and i is just a question. False
i motivated myslef by reminding myself that there is reward for hard work. False
i set my goal to something and do it so in the end i can do what i want False
i took my time and focused and didnt get too stressed False
thinking about going outside False
i kept thinking about the g

In [36]:
gibberish_df

,Anon.Student.Id,input,Invalid Response,Self-Efficacy,Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost \n(outside effort cost),Effort,Extrinsic Rewards/ Self-Consequating,Environmental Control,...,Fear of Failure,Attention,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Notes
0,stu_KAIDW62100,". jjjnnk.nj..mi,.mk.ki.kii..l.kin.",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
1,stu_HMCKN95919,bruhhhh,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
2,sch_NQNWK22810,jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
3,sch_NQNWK22810,3434r 34-gt4ktn'q,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
4,sch_NQNWK22810,idk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
5,sch_NQNWK22810,yayyyyy,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
6,sch_NQNWK22810,idk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
7,sch_NQNWK22810,idk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
8,sch_HTPUV14121,kjh,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9,sch_NQNWK22810,kklj,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN


In [1]:
%pip install lingua-language-detector
from lingua import Language, LanguageDetectorBuilder

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
#Languag Detector - Method 1:
#lingua import: https://github.com/pemistahl/lingua-py
languages = [Language.ENGLISH,Language.SPANISH]
detector = LanguageDetectorBuilder.from_languages(*languages).build()

# Create an empty dataframe to store non-English rows
non_eng_df = pd.DataFrame(columns=text_df_3.columns)

# Iterate through rows and check for non-English texts (confidence_values = 0)
for index, row in text_df_3[:100].iterrows():
    try:
        # Get the confidence values for the language detection
        confidence_values = detector.compute_language_confidence_values(row['input'])
        # Extract the confidence value for English
        english_confidence = next((cv.value for cv in confidence_values if cv.language == Language.ENGLISH), 0)
        # Check if confidence value for English is 0 (i.e., non-English)
        if english_confidence < 0.7:
            # Add the row to the non_eng_df
            non_eng_df = pd.concat([non_eng_df, row.to_frame().T], ignore_index=True)
            # Remove the row from the original dataframe (if needed)
            #text_df_3 = text_df_3.drop(index)
        #print(row['input'], confidence_values.get(Language.ENGLISH, 0))
    except Exception as e:
        print(f"Error processing row {index}: {e}")

In [ ]:
non_eng_df


In [12]:
#generate a new column 'gibberish_detected' to store the gibberish detection results
text_df['gibberish_detected'] = 0  # default column

for index, row in text_df.iterrows():
    try:
        langs = detect_langs(row['input'])

        if len(langs) > 0:
            lang = langs[0].lang
            prob = langs[0].prob

            # Mark as gibberish if not English OR English confidence < threshold
            if langs[0].lang != 'en' and langs[0].prob > 0.8:
                text_df.at[index, 'gibberish_detected'] = 1
            else:
                text_df.at[index, 'gibberish_detected'] = 0
    except:
        text_df.at[index, 'gibberish_detected'] = 1  # Mark as gibberish if detection fails

# compare with human labels
text_df['gibberish_match'] = text_df['gibberish_detected'] == text_df['Invalid Response']
# calculate accuracy
accuracy = text_df['gibberish_match'].mean()
print(f'Gibberish detection accuracy: {accuracy:.2%}')

#pull out the Anon.Student.Id, input, Invalid Response, gibberish_detected, gibberish_match columns for review
text_df_quick_review = text_df[['Anon.Student.Id', 'input', 'Invalid Response', 'gibberish_detected', 'gibberish_match']]

Gibberish detection accuracy: 36.56%


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# True labels (from human-coded data)
y_true = text_df['Invalid Response']
# Predicted labels (from your detection code)
y_pred = text_df['gibberish_detected']

# 1️⃣ Accuracy: How many total predictions were correct
accuracy = accuracy_score(y_true, y_pred)

# 2️⃣ Precision (Of the items flagged as gibberish, how many were actually gibberish), Recall (How many actual gibberish cases were caught), F1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# 3️⃣ Full classification report
print("=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=["Valid", "Gibberish"]))

# 4️⃣ Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\n=== Confusion Matrix ===")
print(cm)

# 5️⃣ Summary output
print(f"\nAccuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-score: {f1:.2%}")

In [14]:
#output the review file
text_df_quick_review.to_excel('/Users/yuchengbao/PycharmProjects/JupyterProject/models/gibberish_detection_review.xlsx', index=False)

In [28]:
text_data_docs = list(nlp.pipe(text_df.input)) #spacy the texts

In [27]:
# preview the text_df
text_df_3

,Anon.Student.Id,input,Invalid Response,Self-Efficacy,Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost \n(outside effort cost),Effort,Extrinsic Rewards/ Self-Consequating,Environmental Control,...,Fear of Failure,Attention,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Notes
0,stu_NWRXR49435,that i will awaly do better then last time,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
2,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
3,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
7,stu_PLFUT13166,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
8,stu_PLFUT13166,focus in classs or ask for help in work from t...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458,sch_NQNWK22810,i told myself to persevere. thats it.,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,NaN
459,sch_NQNWK22810,idk anything about this,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
460,sch_NQNWK22810,i dont know i jjust worked hard,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
461,sch_HTPUV14121,i didnt,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,NaN


After finishing the text-preprocessing, I am going to do the nlp analysis based on the TF-IDF technique

In [38]:
#using the gensim to tokenize the text input for the later NLP analysis
import gensim

tokenized_docs = [list(gensim.utils.tokenize(doc, lower=True)) for doc in text_df_3.input]

In [40]:
# create empty dictionary from gensim library (for the vocabulary)
dictionary = gensim.corpora.Dictionary()
# iteratively add each doc to the bag-of-words corpus
bow_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in tokenized_docs]

print(len(bow_corpus)) #360 texts
print(len(dictionary)) #530 words

360
530


In [42]:
for doc in bow_corpus:
    print([[dictionary[id], freq] for id, freq in doc])

print(len(dictionary)) #530

[['awaly', 1], ['better', 1], ['do', 1], ['i', 1], ['last', 1], ['that', 1], ['then', 1], ['time', 1], ['will', 1]]
[['nan', 1]]
[['nan', 1]]
[['nan', 1]]
[['ask', 2], ['classmates', 1], ['classs', 1], ['fellow', 1], ['focus', 1], ['for', 1], ['from', 1], ['help', 1], ['in', 2], ['or', 2], ['teacher', 1], ['the', 2], ['work', 1]]
[['i', 3], ['help', 1], ['the', 1], ['work', 1], ['am', 1], ['and', 3], ['didnt', 1], ['hecking', 1], ['hit', 1], ['if', 1], ['it', 2], ['make', 1], ['my', 1], ['since', 1], ['solve', 1], ['to', 1], ['use', 1], ['was', 1], ['would', 1]]
[['i', 2], ['if', 1], ['would', 1], ['about', 1], ['by', 1], ['fail', 1], ['happen', 1], ['motivated', 1], ['myself', 1], ['thinking', 1], ['what', 1]]
[['do', 1], ['work', 1], ['to', 1], ['after', 1], ['less', 1]]
[['i', 1], ['and', 1], ['going', 1], ['hard', 1], ['kept', 1], ['worked', 1]]
[['i', 1], ['and', 1], ['it', 1], ['to', 1], ['hard', 1], ['worked', 1], ['done', 1], ['get', 1], ['try', 1]]
[['i', 1], ['get', 1], ['can

In [49]:
import numpy as np

# Get vocab from dictionary
vocab = pd.Series(dictionary.token2id).index

# Sequence of numbers as long as the documents
index = range(len(bow_corpus))

# Create a pandas dataframe full of zeros.
bow_df = pd.DataFrame(data=np.zeros((len(bow_corpus), len(dictionary)), dtype=np.float32),
                  index=index, # row labels
                  columns=vocab) #column labels

# Add each word from the bag-of-words corpus to the matrix.
for idx in index:
  for id, freq in bow_corpus[idx]:
    bow_df.loc[idx,dictionary[id]] = freq
    # updates the specific cell in the DataFrame using by using the document index (row) idx
    # and token ID (column) id, which has been mapped back into the token (word)
    # and sets it to the token's frequency freq

# Transpose the matrix for easy reading
bow_df.T


,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
awaly,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
better,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
do,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
i,1.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,1.0,1.0,...,0.0,0.0,2.0,0.0,1.0,1.0,0.0,2.0,1.0,3.0
last,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
having,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
into,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
mindset,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
persevere,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [50]:
# Calculating Inverse Docmengt Frequency

num_documents = len(bow_df) #get the length of the documents
print(num_documents) #this will be six. Note the df is not transposed

# function of idf
def idf(term):
  documents_containing_term = (bow_df[term] > 0).sum()    # sum of documents containing the term
  return np.log(num_documents+1/documents_containing_term+1)  # divide the number of documents by the num documents with the term and get the log.

# dictionary comprehension to apply the idf function to every word (column) in the bow_df DataFrame
idf = pd.Series({term:idf(term) for term in bow_df.columns}) # create dictionary with key value pair of term:idf(term)
idf

360


awaly        5.891644
better       5.891644
do           5.888940
i            5.888894
last         5.890262
               ...   
having       5.891644
into         5.891644
mindset      5.891644
persevere    5.891644
jjust        5.891644
Length: 530, dtype: float64

In [51]:
#Calculate Term Frequency

tf_df = bow_df.div(bow_df.sum(axis='columns'), axis='rows') # divide each token count by the total token count for the row.
# sum() method specifies the direction along which the sum is calculated
# axis='columns' indicates that the sum is calculated across the columns for each row
# axis='rows' tells the function to perform the division row-wise

tf_df

,awaly,better,do,i,last,that,then,time,will,nan,...,mmyslef,chaing,modivated,selth,food,having,into,mindset,persevere,jjust
0,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.000000
356,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
357,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.142857
358,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000


In [52]:
#Calculate the TF-IDF score

tfidf_df = tf_df.copy().multiply(idf, axis='columns')
# make copy of tf_df that is tf/idf

# Transpose it for readability
tfidf_df.T

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
awaly,0.654627,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
better,0.654627,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
do,0.654327,0.0,0.0,0.0,0.0,0.000000,0.000000,1.177788,0.000000,0.000000,...,0.0,0.0,0.420639,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.368059
i,0.654322,0.0,0.0,0.0,0.0,0.736112,0.981482,0.000000,0.981482,0.654322,...,0.0,0.0,0.841271,0.0,0.267677,0.841271,0.0,1.682541,2.944447,1.104168
last,0.654474,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
having,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.267802,0.000000,0.0,0.000000,0.000000,0.000000
into,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.267802,0.000000,0.0,0.000000,0.000000,0.000000
mindset,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.267802,0.000000,0.0,0.000000,0.000000,0.000000
persevere,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.841663,0.0,0.000000,0.000000,0.000000


Let's try some LDA!